In [1]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
import skweak
import evaluation as evn
import spacy
import torch
import transformers
from transformers import pipeline

In [ ]:
def init_device():
  if torch.cuda.is_available():
    device = torch.device("cuda")
    print("There are %d GPU(s) available." % torch.cuda.device_count())
    print("Using GPU: ", torch.cuda.get_device_name(0))

  else:
    print("No GPU available, using CPU instead.")
    device = torch.device("cpu")
  
init_device()

There are 1 GPU(s) available.
Using GPU:  Tesla K80


In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("conll2002", "nl")

Downloading:   0%|          | 0.00/571k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/194k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset conll2002 downloaded and prepared to /root/.cache/huggingface/datasets/conll2002/nl/1.0.0/a3a8a8612caf57271f5b35c5ae1dd25f99ddb9efb9c1667abaa70ede33e863e5. Subsequent calls will reuse this data.


In [ ]:
raw_datasets["train"].features

{'id': Value(dtype='string', id=None),
 'ner_tags': Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], names_file=None, id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(num_classes=12, names=['Adj', 'Adv', 'Art', 'Conj', 'Int', 'Misc', 'N', 'Num', 'Prep', 'Pron', 'Punc', 'V'], names_file=None, id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [ ]:
def tokenize_function(examples):
  return tokenizer(examples["tokens"], padding = "max_length", truncation = True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched = True)

  0%|          | 0/16 [00:00<?, ?ba/s]

TypeError: ignored

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels = 2)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
from transformers import Trainer
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", evaluation_strategy = "epoch")

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis = -1)
  return metric.compute(predictions = predictions, references = labels)

trainer = Trainer(model = model,
                  args = training_args,
                  train_dataset = small_train_dataset,
                  eval_dataset = small_eval_dataset,
                  compute_metrics = compute_metrics
                  )

trainer.train()
trainer.evaluate()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.506683,0.813000
2,No log,0.324880,0.860000
3,No log,0.522321,0.882000


{'epoch': 3.0,
 'eval_accuracy': 0.882,
 'eval_loss': 0.5223206877708435,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 390197760,
 'eval_runtime': 69.5867,
 'eval_samples_per_second': 14.371}

In [ ]:
nlp = pipeline("ner", "xlm-roberta-base")

sequence = """Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, therefore very close to the Manhattan Bridge which is visible from the window."""

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-st

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
print(nlp(sequence))

[{'word': '▁Hu', 'score': 0.5371246933937073, 'entity': 'LABEL_0', 'index': 1, 'start': 0, 'end': 2}, {'word': 'gging', 'score': 0.5232154130935669, 'entity': 'LABEL_0', 'index': 2, 'start': 2, 'end': 7}, {'word': '▁Face', 'score': 0.5247511267662048, 'entity': 'LABEL_0', 'index': 3, 'start': 8, 'end': 12}, {'word': '▁Inc', 'score': 0.519255518913269, 'entity': 'LABEL_0', 'index': 4, 'start': 13, 'end': 16}, {'word': '.', 'score': 0.5641191005706787, 'entity': 'LABEL_0', 'index': 5, 'start': 16, 'end': 17}, {'word': '▁is', 'score': 0.552219808101654, 'entity': 'LABEL_0', 'index': 6, 'start': 18, 'end': 20}, {'word': '▁a', 'score': 0.540843665599823, 'entity': 'LABEL_0', 'index': 7, 'start': 21, 'end': 22}, {'word': '▁company', 'score': 0.5614894032478333, 'entity': 'LABEL_0', 'index': 8, 'start': 23, 'end': 30}, {'word': '▁based', 'score': 0.5670103430747986, 'entity': 'LABEL_0', 'index': 9, 'start': 31, 'end': 36}, {'word': '▁in', 'score': 0.5356582999229431, 'entity': 'LABEL_0', 'ind